<font size=6> __Connecting to MS SQL Server using sqlalchemy__</font>

<font size=3>Notice: This script will not execute in google colab</font>

<font size=3>Notice: This script is for connecting via Windows Authentication</font>

# Imports

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, update

# Establishing connection

In [2]:
server = 'SHUKI\SQLEXPRESS' 
database = 'tempdb' 

conx_string = f'mssql+pyodbc://{server}/{database}?driver=SQL+Server+Native+Client+11.0'

In [3]:
conx_string

'mssql+pyodbc://SHUKI\\SQLEXPRESS/tempdb?driver=SQL+Server+Native+Client+11.0'

In [4]:
engine = create_engine(conx_string)

In [5]:
# create a SQLAlchemy connection object
conn = engine.connect()

# Executing SQL queries

## Define a function for executing SQL query

In [10]:
def execute_SQL(query_SQL):
    with engine.connect() as conn:
        stmt = text(query_SQL)
        conn.execute(stmt)
        conn.commit()

## Create

In [11]:
create_tryout1_SQL = '''
CREATE TABLE tryout1 (try_id INT, try_name VARCHAR(50))
'''

In [12]:
execute_SQL(create_tryout1_SQL)

In [13]:
# Check your tempdb database to find tryout1 was created (although still empty).

## Insert

In [14]:
insert_tryout1_SQL = '''
INSERT INTO tryout1
VALUES
  (1, 'This is a triumph')
, (2, 'I''m making a note here: Huge success!')
, (3, 'It''s hard to overstate my satisfaction.')
'''

In [15]:
execute_SQL(insert_tryout1_SQL)

In [16]:
# Check your tempdb database to find tryout1 now has data.

## Update

In [17]:
update_tryout1_SQL = '''
UPDATE tryout1
SET try_id = 100
WHERE try_id = 3
'''

In [18]:
execute_SQL(update_tryout1_SQL)

In [19]:
# Check your tempdb database to find try_id 3 is now 100.

## Read

In [20]:
# For reading we would need to do something a little different

In [21]:
select_tryout1_SQL = '''
SELECT *
FROM tryout1
WHERE try_id >= 2
'''

In [22]:
# If we just do that:
tryout1_df = execute_SQL(select_tryout1_SQL)

In [23]:
# We get nothing (because our `execute_SQL` function has no return...)
print(tryout1_df)
print(type(tryout1_df))

None
<class 'NoneType'>


In [24]:
# Instead, we use the `read_sql_query` function.
# Remember - our query has to pass as text()
tryout1_df = pd.read_sql_query(text(select_tryout1_SQL), engine.connect())

In [25]:
tryout1_df

,try_id,try_name
0,2,I'm making a note here: Huge success!
1,100,It's hard to overstate my satisfaction.


In [26]:
# Let's wrap it in a form of a function
def load_SQL(query_SQL):
    res_df = pd.read_sql_query(text(query_SQL), engine.connect())
    return res_df 

In [27]:
tryout1_df_by_func = load_SQL(select_tryout1_SQL)

In [28]:
tryout1_df_by_func

,try_id,try_name
0,2,I'm making a note here: Huge success!
1,100,It's hard to overstate my satisfaction.


## Drop

In [29]:
drop_tryout1_SQL = '''
DROP TABLE tryout1
'''

In [30]:
execute_SQL(drop_tryout1_SQL)

In [31]:
# Check your tempdb database to find tryout1 does not exist.

## Loading a DataFrame into MS SQL Server

In [ ]:
# This is actually extremely easy, we just use the to_sql() method

In [42]:
# Getting some data
ny_baby = pd.read_csv(r'https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv?accessType=DOWNLOAD')

In [43]:
ny_baby.sample(5)

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
19895,2013,FEMALE,WHITE NON HISPANIC,Victoria,100,20
2905,2011,FEMALE,HISPANIC,JAYLIN,13,75
17454,2011,MALE,HISPANIC,LORENZO,14,90
27161,2011,MALE,WHITE NON HISPANIC,GAVIN,53,58
18933,2013,FEMALE,ASIAN AND PACIFIC ISLANDER,Joanna,32,20


In [55]:
# Loading it into SQL
ny_baby.to_sql('ny_baby', con=engine, if_exists='replace', index=False)

300

In [ ]:
# Check your tempdb database to find ny_baby table with all its content.

# Extras

## Improved functions (try / except)

In [39]:
def execute_SQL(query_SQL):
    try:
        with engine.connect() as conn:
            stmt = text(query_SQL)
            conn.execute(stmt)
            conn.commit()
    except Exception as e:
        print(e)

In [40]:
def read_SQL(query_SQL):
    try:
        res_df = pd.read_sql_query(text(query_SQL), engine.connect())
        return res_df
    except Exception as e:
        print(e)